En un medio donde $\mu_s >> \mu_a$ tenemos una fuente puntual que lanza fotones en dirección del eje $z$, el fotón es dispersado si $\xi<\frac{\mu_a}{\mu_s}$, de otra manera es absorbido. $\xi$ es un número aleatorio.

Para muestrear la trayectoria del fotón usamos el método PDF, despejando  de la función de probabilidad dada por la ley de Beer tenemos que: $$x=\frac{ln(\xi)}{\mu_t}$$

donde $\mu_t=\mu_a+\mu_s$, $\mu_a$ y $\mu_s$  son los coeficientes de absorción  y scattering del medio y $\xi $ es un número aleatorio.


Usamos la siguiente fórmula para determinar la dirección del fotón:
 $$cos(\theta) = \left \{ \begin{matrix} 1-2 \xi  & \mbox{si } \mbox{ g=0} \\ \frac{1}{2g}(1+g^2)(\frac{1-g^2}{1-g+2g \xi})^2 & \mbox{si }\mbox{0<g<1}\end{matrix}\right.  $$





$$v_x'=v_x \ cos\theta+ \frac{sen \theta  \ (v_x v_z \ cos\phi-v_y \ sen\phi)}{\sqrt{1-v_z^2}}$$
 
$$v_y'=v_y \cos\theta +\frac{sen \theta  \ (v_y v_z \ cos\phi+ v_x \ sen\phi)}{\sqrt{1-v_z^2}}$$

$$v_z'=v_z \ cos\theta -\sqrt{1-v_z^2} \ sen\theta \ cos\phi$$


$v_x,v_y,v_z$ son los cosenos directores del fotón.

In [1]:
function cosθ(g)
    if g==0
        1-2*rand()
    else
        1/(2*g)*(1+g^2)*(1 - g^2) / (1-g+2*g*rand())
    end
end

cosθ (generic function with 1 method)

In [2]:
function rotacion(g,vx,vy,vz)
    ϕ=rand() 
    cos_θ=cosθ(g)
    senθ = sqrt(abs(1.0 - cos_θ^2)); 
    
    
   
    if vz == 1.0
        vx = senθ * cos(ϕ); #vx,vy,vz son los cosenos directores
            vy = senθ * sin(ϕ); 
        vz = cos_θ      
    
    
   elseif vz == -1.0
        vx = senθ * cosϕ; 
        vy = -senθ * senϕ; 
        vz = -cos_θ; 
     
   else  
        denom = sqrt(abs(1.0 -z^2)); 
        
        ux = senθ*vx*(vz*cosϕ - vy * sin(ϕ) )/denom +vx*cos_θ
        uy = senθ*vy*(vz*cosϕ +vx * sin(ϕ) )/denom +vy*cos_θ
        uz = -denom* senθ * cosϕ + vz* cos_θ; 
         
        
    end


    vx,vy,vz
end

rotacion (generic function with 1 method)

In [3]:
rotacion(.3,2,3,1)

(0.8499407214052979,0.9010828030661945,1.591964022170284)

In [4]:
function un_foton(μs,μa,g)
    vx=0;vy=0;vz=1 #direcciones iniciales
    x=0
    y=0
    z=0 #coordenadas
    u=0;v=0;w=1 #dirección inicial
    μt=μa+μs
    r=0
   
    while true
      t = (-log((rand()+.0000000001)))/μt; 
    x+=t*vx 
    y+=t*vy
    z+=t*vz
            
    r=sqrt(x*x+y*y+z*z)
    
    ξ=rand()
    μ=μa/μs
        if μ<ξ   # Se rompe cuando el fotón se absorbe, la absorción es aleatoria
            break;
    else
            
            vx,vy,vz=rotacion(g,vx,vy,vz)   #el fotón cambia de dirección   
    end
        
    end
    r
end

un_foton (generic function with 1 method)

In [5]:
un_foton(20,1,.3)

0.037560378195475676

Dividimos la distancia en celdas y dependiendo de la distancia por celda acomodamos a los fotones en ellas.

In [6]:
function canales(r_final,δx,r) #ESta función es para dividir las distancias en canales. Se divide r_final en canales  de tamaño δx
    c=[0:δx:r_final]
    j=0
for i in 1:length(c)

    if c[i]==r
        break
        end
        j=i
    end
        j
    end
    
#El canal n vale n*δx
        


canales (generic function with 1 method)

In [7]:
canales(10,1,4)

4

 in depwarn at deprecated.jl:73
 in oldstyle_vcat_warning at ./abstractarray.jl:29
 in canales at In[6]:2
 in include_string at loading.jl:266
 in execute_request_0x535c5df2 at /home/aixa/.julia/v0.4/IJulia/src/execute_request.jl:177
 in eventloop at /home/aixa/.julia/v0.4/IJulia/src/IJulia.jl:141
 in anonymous at task.jl:447
while loading In[7], in expression starting on line 1


Ahora lo implementamos para el paquete de fotones de peso $weigth=1$.

In [8]:
function paquete_foton(μs,μa,g,r_final,δx)
    
albedo = μs / (μs + μa);
    weight=1
    vx=0;vy=0;vz=1 #direcciones iniciales
    x=0
    y=0
    z=0 #coordenadas
    u=0;v=0;w=1 #dirección inicial
    μt=μa+μs # definición
    r=0
    r_h=[0:δx:r_final] #arreglo de posiciones 
    heat=zeros(r_h) #aquí vamos a guardar el calor como función de la posición
   
  
    
    while true
      t = (-log((rand()+.0000000001)))/μt; 
    x+=t*vx 
    y+=t*vy
    z+=t*vz
            
        r=sqrt(x*x+y*y+z*z) #celdas
        
    
        if r > length(heat) #condiciones a la frontera periódicas
            r = length(heat)-1
        end
        
        heat[canales(r_final,δx,r)] = (1.0-albedo)*weight #SE llena el arreglo del calor que depende del albedo como función de la posición.
    ξ=rand()
        μ=μa/μs #Con esta fracción se determina cuando el fotón se absorbe
        
        
        
       
        if weight < 0.001 #russian roulette cuando el paquete es muy pequeño
        
        if μ<ξ   # Se rompe cuando el fotón se absorbe, la absorción es aleatoria
            break;
    else
            
            vx,vy,vz=rotacion(g,vx,vy,vz)   #el fotón cambia de dirección   
    end
        end
        
        weight /= 0.1  #A cada paso diminuyen los paquetes de fotones
        
    end
    heat
end


paquete_foton (generic function with 1 method)

In [ ]:
paquete_foton(20,2,.3,1,.1)

 in depwarn at deprecated.jl:73
 in oldstyle_vcat_warning at ./abstractarray.jl:29
 in paquete_foton at In[8]:12
 in include_string at loading.jl:266
 in execute_request_0x535c5df2 at /home/aixa/.julia/v0.4/IJulia/src/execute_request.jl:177
 in eventloop at /home/aixa/.julia/v0.4/IJulia/src/IJulia.jl:141
 in anonymous at task.jl:447
while loading In[9], in expression starting on line 1
 in depwarn at deprecated.jl:73
 in oldstyle_vcat_warning at ./abstractarray.jl:29
 in canales at In[6]:2
 in paquete_foton at In[8]:30
 in include_string at loading.jl:266
 in execute_request_0x535c5df2 at /home/aixa/.julia/v0.4/IJulia/src/execute_request.jl:177
 in eventloop at /home/aixa/.julia/v0.4/IJulia/src/IJulia.jl:141
 in anonymous at task.jl:447
while loading In[9], in expression starting on line 1
 in depwarn at deprecated.jl:73
 in oldstyle_vcat_warning at ./abstractarray.jl:29
 in canales at In[6]:2
 in paquete_foton at In[8]:30
 in include_string at loading.jl:266
 in execute_request_0x535c

No sé por qué no corre

Dividimos la distancia en celdas y dependiendo de la distancia por celda acomodamos a los fotones en ellas.